# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [4]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import numpy as np
import pandas as pd
import sklearn as sk

[nltk_data] Downloading package stopwords to /home/jean-
[nltk_data]     michel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [6]:
X, y = X[:], y[:]

In [7]:
tokenized_documents = tokenizer(X.copy())

Number of tokens:  0
Number of sentences:  0


In [ ]:
X, vect = vectorizer(tokenized_documents)

: 

In [ ]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(160000, 300) (40000, 300) (160000,) (40000,)
[0.4905342  0.53652514 0.60810707 0.53445703 0.47397945 0.63044243
 0.77788866 0.57545352 0.62664844 0.49715616 0.59787111 0.406304
 0.55857397 0.35807929 0.62644771 0.63388435 0.63886314 0.39206989
 0.6117431  0.46993149 0.55822945 0.54868934 0.41539238 0.56931953
 0.66263493 0.40275917 0.60358226 0.30975076 0.40173751 0.57448015
 0.49435486 0.48770446 0.5582401  0.54829421 0.40585013 0.21431948
 0.70002451 0.50918965 0.53844196 0.53878083 0.6792521  0.6003442
 0.63211479 0.41174714 0.31062602 0.59575013 0.4498044  0.63131857
 0.66233253 0.41354765 0.60187123 0.37521605 0.53285976 0.32667534
 0.43353285 0.38850996 0.58945693 0.49794166 0.65526871 0.44370182
 0.48311241 0.58029061 0.72859698 0.44735472 0.5844671  0.63727247
 0.32446904 0.60332818 0.51567436 0.52645433 0.28422345 0.57145645
 0.5578231  0.53274498 0.53082822 0.54409617 0.54493374 0.56921478
 0.70193791 0.53333375 0.61360615 0.4092376  0.62622802 0.67085593
 0.45017457 0.40164

# Feed Forward Neural Network
### Model starts here

In [ ]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU
from sklearn import metrics
import datetime
import os

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
    ModelCheckpoint('model_best.keras', save_best_only=True, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=0.00001)
]

In [ ]:
def batch_generator(X, y, batch_size, num_classes):
    num_samples = X.shape[0]
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end]
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield X_batch, y_batch

# Setup the model
inputs = Input(shape=(X_train.shape[1],))
x = Dense(256)(inputs)
x = LeakyReLU()(x)
x = Dropout(0.1)(x)
x = Dense(128)(x)
x = LeakyReLU()(x)
residual = Dense(64)(x)
x = LeakyReLU()(residual)

x = Dense(64)(x)
x = LeakyReLU()(x)
x = Dropout(0.1)(x)
x = Add()([x, residual])
x = Dense(32)(x)
x = LeakyReLU()(x)
x = Dense(16)(x)
x = LeakyReLU()(x)

outputs = Dense(np.max(y_train) + 1, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

# Define the generator
train_generator = batch_generator(X_train, y_train, 32, np.max(y_train) + 1)


In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=int(np.ceil(X_train.shape[0] / 32)),
    epochs=80,
    callbacks=callbacks,
)

Epoch 1/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.6360 - loss: 1.0801 - learning_rate: 0.0010
Epoch 2/80
 144/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6516 - loss: 0.9790

/home/jean-michel/Epita/nlp/.venv/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/jean-michel/Epita/nlp/.venv/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/home/jean-michel/Epita/nlp/.venv/lib/python3.10/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6543 - loss: 0.9604 - learning_rate: 0.0010
Epoch 3/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6573 - loss: 0.9439 - learning_rate: 0.0010
Epoch 4/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.6587 - loss: 0.9358 - learning_rate: 0.0010
Epoch 5/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6609 - loss: 0.9280 - learning_rate: 0.0010
Epoch 6/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6627 - loss: 0.9215 - learning_rate: 0.0010
Epoch 7/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6627 - loss: 0.9177 - learning_rate: 0.0010
Epoch 8/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.6634 - loss: 0.9147 - learning_rate: 0.0010
Epoch 9/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6640 - loss: 0.9117 - learning_rate: 0.0010
Epoch 10/80
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6655 - loss: 0.9085 - learning_rate: 0.0010
Ep

In [ ]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1)

In [ ]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print("Confusion Matrix:")
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

print("\nClassification Report:")
print(metrics.classification_report(y_test, predictions))

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 723us/step
Confusion Matrix:
[[ 2189    27    63    13  1368]
 [  630    72   160    47  1136]
 [  476    51   293   196  2152]
 [  255    21   136   308  5118]
 [  492    13    67   140 24577]]

Classification Report:
              precision    recall  f1-score   support

           1       0.54      0.60      0.57      3660
           2       0.39      0.04      0.06      2045
           3       0.41      0.09      0.15      3168
           4       0.44      0.05      0.09      5838
           5       0.72      0.97      0.82     25289

    accuracy                           0.69     40000
   macro avg       0.50      0.35      0.34     40000
weighted avg       0.62      0.69      0.60     40000



In [ ]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config3_feedforward_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

In [ ]:
# Test the model
test_sentences = [
    'This is a good product',
    'This is a bad product',
    'This is a product',
    'This is a very good product',
    'This is a very bad product',
    'That was bad'
]

for sentence in test_sentences:
    test_tokenized = tokenizer([sentence])
    test_vec = vect.transform(test_tokenized).toarray()
    test_vec_lstm = test_vec.reshape(1, -1)
    result = model.predict(test_vec_lstm)
    predicted_class = result.argmax()
    predicted_score = predicted_class
    print(f"Test sentence: {sentence}")
    print(f"Predicted score: {predicted_score}")
    print()

NameError: name 'tokenizer' is not defined